# 🎨 **Stable Diffusion** 🎨 

Hi Rachel 👋

### Log in to Hugging Face

In [ ]:
!pip install diffusers==0.2.4

from google.colab import output
output.enable_custom_widget_manager()

from huggingface_hub import notebook_login
notebook_login()

### Setup

In [ ]:
!nvidia-smi

!pip install transformers scipy ftfy
!pip install "ipywidgets>=7,<8"

import torch
from torch import autocast

from diffusers import StableDiffusionPipeline

# make sure you're logged in with `huggingface-cli login`
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", revision="fp16", torch_dtype=torch.float16, use_auth_token=True)  

pipe = pipe.to("cuda")

# Define the image grid if we need it later
from PIL import Image
def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

### Generate images

#### Single image

And this one is the only one so far that outputs the seed, so we can regenerate it.

In [ ]:
generator = torch.Generator("cuda")

prompt = "a LinkedIn profile picture of a koala in a suit"
steps = 50

with autocast("cuda"):
  image = pipe(prompt, num_inference_steps=steps, height=512, width=512, generator=generator)["sample"][0]

print("Seed:")
print(generator.initial_seed())

image

#### Three at once

In [ ]:
num_images = 3
prompt = ["a photograph of an astronaut riding a horse"] * num_images

with autocast("cuda"):
  images = pipe(prompt)["sample"]

grid = image_grid(images, rows=1, cols=3)
grid

#### A grid of 6

In [ ]:
num_cols = 3
num_rows = 2

prompt = ["a photograph of an astronaut riding a horse"] * num_cols

all_images = []
for i in range(num_rows):
  with autocast("cuda"):
    images = pipe(prompt)["sample"]
  all_images.extend(images)

grid = image_grid(all_images, rows=num_rows, cols=num_cols)
grid

#### Using a specific seed

In [ ]:
prompt = "oil painting of a Canadian goose with a gold sepia filter"
seed = 1591226903
generator = torch.Generator("cuda").manual_seed(seed)

with autocast("cuda"):
  image = pipe(prompt, generator=generator, height=512, width=512, num_inference_steps=50)["sample"][0]

image